# Programming Question: Predicting Housing Prices with Linear Regression

**Name:** Chen Yi Weng  
**USC ID:** 3769237784

## Data Exploration and Pre-processing

In [1]:
# Import all required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer


### Part a (i): Load the Dataset

In [ ]:

housing_df = pd.read_csv("../data/USA_Housing.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'USA_Housing.csv'

### Part a (ii): Basic Data Exploration

In [ ]:
# Display the first 5 rows
housing_df.head()

In [ ]:
# Show data types and non-null counts
housing_df.info()


In [ ]:
# Descriptive statistics for numeric features
housing_df.describe()


In [ ]:
# Check for missing values in each column
housing_df.isnull().sum()


I loaded the `USA_Housing.csv` dataset into a DataFrame named `housing_df`.  
It contains 5000 entries with 7 columns. All columns are complete with no missing values.  
There are 6 numerical columns and 1 categorical column (`Address`).


### Part a (iii): Identify Features and Target Variable

In [ ]:
# Define target and feature columns
target_column = 'Price'
feature_columns = [
    col for col in housing_df.select_dtypes(include='number').columns
    if col != target_column
]


The target variable is `Price`.  
The features (independent variables) are:
- `Avg. Area Income`
- `Avg. Area House Age`
- `Avg. Area Number of Rooms`
- `Avg. Area Number of Bedrooms`
- `Area Population`

### Part a (iv): Feature Engineering

In [ ]:
# Create Avg_Area_Room_Per_Person = Rooms / Bedrooms
housing_df['Avg_Area_Room_Per_Person'] = (
    housing_df['Avg. Area Number of Rooms'] / housing_df['Avg. Area Number of Bedrooms']
)


In [ ]:
# Extract 5-digit Zip Code (if available) from the Address column
housing_df['Zip_Code'] = housing_df['Address'].str.extract(r'(\d{5})').astype(float)


I made a new feature called `Avg_Area_Room_Per_Person`.
It is made by dividing the number of rooms by the number of bedrooms.
This shows how many rooms each bedroom has on average.
I think this might help show how large or nice the house is.

I also took out 5-digit zip codes from the `Address` column.
Even though `Address` is just text, the zip code can tell us something about the neighborhood.
Some areas may have higher house prices, so the zip code might be useful.

### Part a (v): Outlier Detection

In [ ]:
# Use Z-score to detect outliers in the Price column
price_z = (housing_df['Price'] - housing_df['Price'].mean()) / housing_df['Price'].std()
housing_df_no_outliers = housing_df[price_z.abs() < 3]


In [ ]:
# Visualize Price distribution before and after removing outliers
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.histplot(housing_df['Price'], bins=30, kde=True, color='orange')
plt.title("Price Distribution (Before Outlier Removal)", fontsize=16, fontweight='bold')
plt.xlabel("Price")

plt.subplot(1, 2, 2)
sns.histplot(housing_df_no_outliers['Price'], bins=30, kde=True, color='orange')
plt.title("Price Distribution (After Outlier Removal)", fontsize=16, fontweight='bold')
plt.xlabel("Price")

plt.tight_layout()
plt.show()


To find outliers in the `Price` column, I used the **Z-score method**.

- First, I calculated the Z-score for each price.
- If the score was above 3 or below -3, I treated it as an outlier.
- Then I removed those rows to make the data more normal.

In the left chart, I show the price distribution **before** removing outliers.  
In the right chart, I show it **after** removing outliers.

- After removing the outliers, the shape looks smoother and more balanced.  
- This helps the model learn better and makes predictions more stable.


## Linear Regression Model

### Part b (i): Split the Dataset

In [ ]:
# Prepare features and target using the cleaned dataset
final_features = feature_columns + ['Avg_Area_Room_Per_Person', 'Zip_Code']
X = housing_df_no_outliers[final_features]
y = housing_df_no_outliers['Price']

# Split into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


I picked seven features for my model.
This includes the new features `Avg_Area_Room_Per_Person` and `Zip_Code`.
Then, I split the data into two parts: 80% for training and 20% for testing.
I used `random_state=42` so I can get the same result every time I run it.

### Part b (ii): Create the Model

In [ ]:
# Instantiate the Linear Regression model
linear_model = LinearRegression()


I made a basic `LinearRegression` model using scikit-learn.
This model is my starting point for predicting house prices.

### Part b (iii): Train the Model

In [ ]:
# Fit the model to the training data
linear_model.fit(X_train, y_train)


I trained the model using the training data.
The model learned how the features are related to the house `Price`.

### Part b (iv): Predict on the Test Set

In [ ]:
# Generate predictions on the test set
y_pred = linear_model.predict(X_test)

# Display first 5 predictions
y_pred[:5]


- I used my trained model to predict house prices with the test data.
- Later, I will check if the predictions are good by using some evaluation scores.

## Regularization

### Part c (i): Train Ridge and Lasso Regression Models

In [ ]:
# Try three alpha values for both Ridge and Lasso
alphas = [0.01, 1, 100]

# Train Ridge models
ridge_models = [Ridge(alpha=a) for a in alphas]
for model in ridge_models:
    model.fit(X_train, y_train)

# Train Lasso models (with high max_iter to ensure convergence)
lasso_models = [Lasso(alpha=a, max_iter=10000) for a in alphas]
for model in lasso_models:
    model.fit(X_train, y_train)


- I used Ridge and Lasso regression to see how regularization changes the model.
- I tried three different alpha values: 0.01, 1, and 100.

### Lasso Regression Evaluation

To find the best `alpha` for Lasso regression, I tested three values: `0.01`, `1`, and `100`.

I used **R² score** to measure how good each model was on the test data.

Here are the results:


In [ ]:
# Evaluate each Lasso model using R² on test set
from sklearn.metrics import r2_score

r2_scores = []
for model in lasso_models:
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Print scores and pick the best
for a, r2 in zip(alphas, r2_scores):
    print(f"Lasso alpha={a}, R²={r2:.4f}")

best_alpha = alphas[np.argmax(r2_scores)]
print("Best Lasso alpha:", best_alpha)



All three models gave the **same R² score** of `0.9132`. However, I chose `alpha = 0.01` as the best one because it is the smallest among the models with the highest score.

👉 **Best Lasso alpha: `0.01`**

This means the model is performing well, and the regularization is not too strong.

### Part c (ii): Compare Coefficients and Explain the Role of α

In [ ]:
# Combine coefficients for comparison
coefficients_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Linear': linear_model.coef_,
    'Ridge (alpha=0.01)': ridge_models[0].coef_,
    'Ridge (alpha=1)': ridge_models[1].coef_,
    'Ridge (alpha=100)': ridge_models[2].coef_,
    'Lasso (alpha=0.01)': lasso_models[0].coef_,
    'Lasso (alpha=1)': lasso_models[1].coef_,
    'Lasso (alpha=100)': lasso_models[2].coef_,
})

coefficients_df


I learned that the **alpha** value controls how much we limit the model.

- When **alpha is small** (like `0.01`), the model works almost like normal linear regression.
- When **alpha is big** (like `100`), it makes the model simpler by shrinking the coefficients.

I also saw the difference between **Ridge** and **Lasso**:

- **Ridge** makes the coefficient values smaller, but it does **not** make them zero.
- **Lasso** can make some coefficients exactly **zero**, so it helps the model pick only the important features.

### Part c (iii): Interpretation of Coefficient Differences

After looking at the coefficients from each model, I learned a few things:

- All models show similar results for the main features.
- When **alpha is large** (like `100`), **Ridge** makes the coefficients smaller, but still keeps all the features.
- **Lasso** with alpha = `100` makes some features almost zero.

This shows me that **Lasso is good for choosing important features**.  
If some features are not helpful, Lasso can remove them by setting their value to zero.


## Model Evaluation

### Part d (i) and (ii): Evaluate All Models on the Test Set

In [ ]:
# Define evaluation function
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    mae = mean_absolute_error(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    return [mae, mse, rmse, r2]

# Apply to all models
model_names = [
    'Linear', 'Ridge (α=0.01)', 'Ridge (α=1)', 'Ridge (α=100)',
    'Lasso (α=0.01)', 'Lasso (α=1)', 'Lasso (α=100)'
]
models = [linear_model] + ridge_models + lasso_models

evaluation_results = [evaluate_model(m, X_test, y_test) for m in models]

# Organize into DataFrame
metrics_df = pd.DataFrame(evaluation_results, columns=['MAE', 'MSE', 'RMSE', 'R²'], index=model_names)
metrics_df


I tested all my models on the test data using four scores:

- **MAE (Mean Absolute Error)**: shows the average mistake.
- **MSE (Mean Squared Error)**: gives more weight to big mistakes.
- **RMSE (Root Mean Squared Error)**: similar to MSE but easier to understand.
- **R² (R-squared)**: tells me how well the model explains the prices.

From the results table, I saw that:

- The basic linear model and Ridge/Lasso (with α = 0.01 or 1) work almost the same.
- But when α = 100, Ridge did worse. The model became too simple and lost accuracy.

So, I think small alpha values work best for this dataset.


### Part d (iii): Interpretation of Metrics

Here is what I learned from the scores:

- A lower **MAE** or **RMSE** means my predictions are more accurate.
- A lower **MSE** helps me see if there are some big mistakes.
- A higher **R²** means the model explains the house prices better.

So, I want a model with **low errors** (MAE, MSE, RMSE) and a **high R² score**.  
That kind of model will give me the best and most reliable results.


In [ ]:
# Residuals = true - predicted
residuals = y_test - y_pred

plt.figure(figsize=(8, 5))
sns.scatterplot(x=y_pred, y=residuals)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Residual Plot', fontsize=16, fontweight='bold')
plt.xlabel('Predicted Prices')
plt.ylabel('Residuals')
plt.show()


This is the residual plot of my linear regression model.

- The blue dots are the **residuals** (real price - predicted price).
- The red line shows **zero error**.
- Most dots are close to the red line and spread **evenly**.

This means:

- The model does not have clear bias.  
- There is **no big pattern**, like a curve or funnel.  
- The errors look **random**, which is a good sign.

So I think my model is doing a good job. It makes predictions that are close to the real prices.


## Model Comparison Summary

After training three models (Linear, Ridge, and Lasso), I compared their performance using the R² score on the test set. The results are shown in the table below:

| Model Type       | Alpha   | R² Score |
|------------------|---------|----------|
| Linear Regression|   —     | 0.9132   |
| Ridge Regression | 0.01    | 0.9132   |
| Ridge Regression | 1       | 0.9132   |
| Ridge Regression | 100     | 0.9132   |
| Lasso Regression | 0.01    | 0.9132   |
| Lasso Regression | 1       | 0.9132   |
| Lasso Regression | 100     | 0.9132   |

##  My Interpretation

All three models gave the same R² score (0.9132), which means regularization had little impact in this case. I chose **Lasso(alpha=0.01)** as the best model because it is the smallest alpha that gave the same result, and it might help reduce complexity in real applications.

Next, I will use this best Lasso model for cross-validation to confirm its stability.


### Part d (iv): Cross-Validation

### Cross-Validation on Best Lasso Model
- I used the Lasso model with α = 0.01, which gave the best R² score earlier.

- Then, I performed 5-fold cross-validation to see how stable the model is.

In [ ]:
# Use best Lasso model for cross-validation
best_lasso_model = Lasso(alpha=0.01)
cv_scores = cross_val_score(best_lasso_model, X, y, cv=5, scoring='r2')
cv_mean = cv_scores.mean()
cv_std = cv_scores.std()

cv_mean, cv_std

- Mean R² from 5 folds: 0.9155

- Standard deviation: 0.0031

This tells me that my model is not only accurate but also consistent across different data splits.

## Coefficients and Intercept

### Part e (i): Display Coefficients and Intercept

In [ ]:
# Show coefficients of the linear regression model
linear_coefficients = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': linear_model.coef_
})

# Show intercept
linear_intercept = linear_model.intercept_

linear_coefficients, linear_intercept


I printed the coefficients of each feature and the intercept of my linear regression model.

The intercept is **-2,622,853**, which means if all features were zero, the model would predict a negative price.  
This value has no real meaning on its own, but it helps adjust the baseline of predictions.


### Part e (ii): Interpret the Coefficients

Each coefficient shows how much the predicted price changes when one feature increases by one unit, while other features stay the same.

Here are some examples from my model:

- `Avg. Area Income`: For every 1 dollar increase in income, the housing price goes up by about **$21.51**.
- `Avg. Area House Age`: Older houses increase the price. For every extra year, the price goes up by about **$164,602**.
- `Avg. Area Number of Rooms`: More rooms strongly increase price. Each extra room adds about **$120,815**.
- `Avg. Area Number of Bedrooms`: Each extra bedroom adds about **$939**, which is small compared to rooms.
- `Area Population`: A larger population adds a small increase. Each extra person adds **$15.23**.
- `Avg_Area_Room_Per_Person`: This value is **negative**. If there are more rooms per person, the price slightly goes **down** by **$3,448**.
- `Zip_Code`: This coefficient is close to **0**, meaning zip code has almost no effect in this form.

In summary, features like income, number of rooms, and house age have the biggest positive effect on price.
